## Бэггинг и случайный лес
Загрузите датасет `digits` с помощью функции `load_digits` из `sklearn.datasets` и подготовьте матрицу признаков `X` и ответы на обучающей выборке `y` (вам потребуются поля `data` и `target` в объекте, который возвращает `load_digits`).

In [1]:
from sklearn.datasets import load_digits
from sklearn import ensemble, model_selection, learning_curve, metrics, tree
import numpy as np
import pandas as pd
%pylab inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
digits = load_digits()
X = digits.data
y = digits.target
X.shape

(1797, 64)

In [3]:
def write_array(answers, filename):
    with open(filename, "w") as fout:
        fout.write(" ".join([str(num) for num in answers]))
        
def write_ans(answer, filename):
    with open(filename, "w") as fout:
        fout.write(answer)

## Задание 1\.

Для оценки качества далее нужно будет использовать `cross_val_score` из `sklearn.cross_validation` с параметром `cv=10`. Эта функция реализует k-fold cross validation c `k` равным значению параметра `cv`. Мы предлагаем использовать `k=10`, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и `k=5`. Функция `cross_val_score` будет возвращать `numpy.ndarray`, в котором будет `k` чисел - качество в каждом из `k` экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод `.mean()` у массива, который возвращает `cross_val_score`.

С небольшой вероятностью вы можете натолкнуться на случай, когда полученное вами качество в каком-то из пунктов не попадет в диапазон, заданный для правильных ответов - в этом случае попробуйте перезапустить ячейку с `cross_val_score` несколько раз и выбрать наиболее «типичное» значение. Если это не помогает, то где-то была допущена ошибка.

Если вам захочется ускорить вычисление `cross_val_score` - можете попробовать использовать параметр `n_jobs`, но будьте осторожны: в одной из старых версий sklearn была ошибка, которая приводила к неверному результату работы `cross_val_score` при задании `n_jobs` отличным от `1`. Сейчас такой проблемы возникнуть не должно, но проверить, что все в порядке, не будет лишним.

Создайте `DecisionTreeClassifier` с настройками по умолчанию и измерьте качество его работы с помощью `cross_val_score`. Эта величина и будет ответом в пункте `1`.

In [4]:
cv = 10
dt_classifier = tree.DecisionTreeClassifier()
score_dt = model_selection.cross_val_score(dt_classifier, X, y, cv = cv)
print(score_dt.mean())
write_ans(str(score_dt.mean()), 'ans_1.txt')

0.828003786801


## Задание 2\. 

Воспользуйтесь `BaggingClassifier` из `sklearn.ensemble`, чтобы обучить бэггинг над `DecisionTreeClassifier`. Используйте в `BaggingClassifier` параметры по умолчанию, задав только количество деревьев равным `100`.

Качество классификации новой модели – ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [5]:
bagging_defolt = ensemble.BaggingClassifier(n_estimators=100)
score_bagging_defolt = model_selection.cross_val_score(bagging_defolt, X, y, cv = cv)
print(score_bagging_defolt.mean())
write_ans(str(score_bagging_defolt.mean()), 'ans_2.txt')

0.927180897853


## Задание 3\. 
Теперь изучите параметры `BaggingClassifier` и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех $d$ признаках, а на $\sqrt d$ случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. 

Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [6]:
bagging_d8 = ensemble.BaggingClassifier(n_estimators=100, max_features=8)
score_bagging_d8 = model_selection.cross_val_score(bagging_d8, X, y, cv = cv)
print(score_bagging_d8.mean())
write_ans(str(score_bagging_d8.mean()), 'ans_3.txt')

0.931487084738


## Задание 4\. 
Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в `BaggingClassifier` и добавить его в `DecisionTreeClassifier`. Какой параметр за это отвечает, можно понять из документации `sklearn`, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же $\sqrt d$ признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [7]:
bagging = ensemble.BaggingClassifier(tree.DecisionTreeClassifier(max_features=8), n_estimators=100)
score_bagging = model_selection.cross_val_score(bagging, X, y, cv = cv)
print(score_bagging.mean())
write_ans(str(score_bagging.mean()), 'ans_4.txt')    

0.952776546692


## Задание 5\. 

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с `RandomForestClassifier` из `sklearn.ensemble`. 

Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

In [8]:
random_forest = ensemble.RandomForestClassifier(n_estimators=100, max_features=8)
score_random_forest = model_selection.cross_val_score(random_forest, X, y, cv = cv)
print(score_random_forest.mean())

0.951137071876


На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1. Случайный лес сильно переобучается с ростом количества деревьев
2. При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев
3. С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.
4. При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.
5. При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.
6. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.
7. При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг друга).

In [9]:
random_forest_d5 = ensemble.RandomForestClassifier(n_estimators=100, max_features=5)
score_random_forest_d5 = model_selection.cross_val_score(random_forest_d5, X, y, cv = cv)
print('5 features:', score_random_forest_d5.mean())

random_forest_d50 = ensemble.RandomForestClassifier(n_estimators=100, max_features=50)
score_random_forest_d50 = model_selection.cross_val_score(random_forest_d50, X, y, cv = cv)
print('50 features:', score_random_forest_d50.mean())

5 features: 0.951713457717
50 features: 0.934334487156


In [10]:
random_forest_deep5 = ensemble.RandomForestClassifier(n_estimators=100, max_features=8, max_depth=5)
score_random_forest_deep5 = model_selection.cross_val_score(random_forest_deep5, X, y, cv = cv)
print('max_depth=5', score_random_forest_deep5.mean())

random_forest_deep_max = ensemble.RandomForestClassifier(n_estimators=100, max_features=8)
score_random_forest_deep_max = model_selection.cross_val_score(random_forest_deep_max, X, y, cv = cv)
print('max_depth = infinity', score_random_forest_deep_max.mean())

max_depth=5 0.918235104834
max_depth = infinity 0.95222409995


In [11]:
write_array([2, 3, 4, 7], 'ans_5.txt')